In [ ]:
!pip install tensorboardX
!pip install Unidecode
! pip install phonemizer
!sudo apt-get install espeak
!pip install soundfile
# install espeak backend if you like to use phonemes instead of raw characters
!sudo apt-get install espeak
!pip install soundfile
!pip install pysbd
!pip install pydub
!pip install pydub1

     |████████████████████████████████| 317kB 9.3MB/s 
     |████████████████████████████████| 245kB 8.9MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 194kB 13.3MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/TTS-master 

/content/drive/MyDrive/TTS-master


In [1]:
import os
import importlib
import random
import librosa
import torch
from TTS.speaker_encoder.model import SpeakerEncoder
import numpy as np
from tqdm import tqdm
from TTS.tts.utils.speakers import save_speaker_mapping, load_speaker_mapping

# you may need to change this depending on your system
os.environ['CUDA_VISIBLE_DEVICES']='0'


from TTS.tts.utils.speakers import save_speaker_mapping, load_speaker_mapping

from TTS.utils.audio import AudioProcessor
from TTS.utils.io import load_config
import pandas as pd

In [2]:

MODEL_PATH = "/home/Ciro/Desktop/ItalianMoziillaTTS/TTS-master/TTS/speaker_encoder/best_model.pth.tar" #qui ci va il tuo pre_trained speaker encoder
CONFIG_PATH = "/home/Ciro/Desktop/ItalianMoziillaTTS/TTS-master/TTS/speaker_encoder/config.json" #qui ci va il tuo file di configurazione con cui hai allenato il tuo encoder

USE_CUDA = True

In [3]:


def ljspeech(root_path, meta_file):
    """Normalizes the Nancy meta data file to TTS format"""
    txt_file = os.path.join(root_path, meta_file)
    items = []
    
    with open(txt_file, 'r') as ttf:
        for line in ttf:
            cols = line.split('|')
            speaker_name = cols[0]
            wav_file = os.path.join(root_path, cols[1])
            text = cols[2]
            items.append([text, wav_file, speaker_name])
    return items



In [21]:
items


NameError: name 'items' is not defined

In [4]:
meta_data = list(ljspeech("/home/Ciro/Desktop/ItalianMoziillaTTS/TTS-master/TTS/tts/datasets/output_dataset_it_adhoc/sansone", "metadata_train.csv"))
meta_data += list(ljspeech("/home/Ciro/Desktop/ItalianMoziillaTTS/TTS-master/TTS/tts/datasets/output_dataset_it_adhoc/sansone", "metadata_val.csv"))
 #questo è un file cvs: contiene per ogni recording la trascrizione, 
#path alla registrazione e lo spekaer_id. 

In [5]:
len(meta_data)

369

In [9]:

#qui carichi il tuo speaker encoder 
c = load_config(CONFIG_PATH)
ap = AudioProcessor(**c['audio'])

model = SpeakerEncoder(**c.model)
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu') )['model'])
model.eval()
if USE_CUDA:
    model.cuda()




 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.98
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:256
 | > win_length:1024


In [7]:
sr = 22050

In [10]:

#qui estrai gli embedding 
embeddings_dict = {}
len_meta_data= len(meta_data)

for i in tqdm(range(len_meta_data)):
    speaker_id, sentence, wav_file = meta_data[i]
    wav_file_name = os.path.basename(wav_file)
    print(speaker_id)
    wav_file = os.path.join("/home/Ciro/Desktop/ItalianMoziillaTTS/TTS-master/TTS/tts/datasets/output_dataset_it_adhoc/sansone/", wav_file) #path ai tuou audio perche ne hai bisogno per estrarre embedding. 
    x, sr = librosa.load(wav_file, sr=sr)
    mel_spec = ap.melspectrogram(x).T
    mel_spec = torch.FloatTensor(mel_spec[None, :, :])
    if USE_CUDA:
        mel_spec = mel_spec.cuda()
    embedd = model.compute_embedding(mel_spec).cpu().detach().numpy().reshape(-1)
    embeddings_dict[wav_file_name] = [embedd,speaker_id]
    

    

  0%|          | 1/369 [00:00<00:38,  9.46it/s]

10459

10459



  1%|          | 3/369 [00:00<00:44,  8.27it/s]

10459

10459

10459



  1%|▏         | 5/369 [00:00<00:33, 10.94it/s]

...

 99%|█████████▉| 366/369 [00:42<00:00,  9.58it/s]

10459

10459



100%|█████████▉| 368/369 [00:42<00:00,  7.90it/s]

10459


100%|██████████| 369/369 [00:42<00:00,  8.60it/s]


In [46]:
# create and export speakers.json
speaker_mapping = {sample: {'name': embeddings_dict[sample][1], 'embedding':embeddings_dict[sample][0].reshape(-1).tolist()} for i, sample in enumerate(embeddings_dict.keys())}
save_speaker_mapping("/home/Ciro/Desktop/ItalianMoziillaTTS/TTS-master/TTS/speaker_encoder", speaker_mapping)

In [47]:
speaker_mapping_load = load_speaker_mapping("/home/Ciro/Desktop/ItalianMoziillaTTS/TTS-master/TTS")
assert speaker_mapping == speaker_mapping_load
print("The file speakers.json has been exported to ", "/content/drive/MyDrive/Test_Locale", ' with ', len(embeddings_dict.keys()), ' speakers')

AssertionError: 